In [1]:
import os
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. 配置MLflow客户端
# -----------------------------------
# 告诉MLflow客户端，Tracking Server在哪里
# (使用Docker网络内部的服务名)
os.environ["MLFLOW_TRACKING_URI"] = "http://mlflow-server:5000"

# 告诉MLflow客户端，S3(MinIO)产物存储在哪里
# (这是让客户端能直接上传到MinIO的关键)
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://minio:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"

print(f"MLFLOW_TRACKING_URI set to: {os.environ['MLFLOW_TRACKING_URI']}")

MLFLOW_TRACKING_URI set to: http://mlflow-server:5000


In [ ]:

# 2. 设置实验
# -----------------------------------
# 实验将按名称自动创建
experiment_name = "my_first_ml_experiment"
mlflow.set_tracking_uri("http://mlflow-server:5000")
mlflow.set_experiment(experiment_name)

print(f"Using MLflow experiment: {experiment_name}")

# 3. 准备数据和参数
# -----------------------------------
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 我们要试验的超参数
model_params = {
    "C": 0.003927893213212713,
    "solver": "saga"
}

print("Data and parameters are ready.")

# 4. 开始一个MLflow "Run" (一次实验)
# -----------------------------------
with mlflow.start_run() as run:
    print(f"Starting run: {run.info.run_id}")

    # 5. 训练模型
    # -----------------------------------
    model = LogisticRegression(C=model_params["C"], solver=model_params["solver"])
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # 6. 记录到MLflow服务器
    # -----------------------------------
    print("Logging to MLflow...")

    # a. 记录参数
    mlflow.log_params(model_params)
    print(f"Logged params: {model_params}")

    # b. 记录指标
    mlflow.log_metric("accuracy", accuracy)
    print(f"Logged metric 'accuracy': {accuracy}")

    # c. 记录模型文件
    # (这会自动将模型保存并上传到MinIO)
    # "model" 是产物在MinIO中的子目录名
    mlflow.sklearn.log_model(model, name="model")
    print("Logged model to MinIO.")

print("\n--- Experiment Run Finished! ---")

In [ ]:
import time

# --- 我们假装上一单元格刚运行完 ---
# run_id 应该是上一个单元格中 mlflow.start_run() 产生的
# (如果单元格已停止，你需要去UI上手动复制那个 run_id)
# 假设 run_id 存储在变量 run.info.run_id 中
run_id = run.info.run_id 
model_uri = f"runs:/{run_id}/model"

# --- 这是新内容：注册模型 ---
model_name = "iris_logistic_regression"

print(f"Registering model '{model_name}' from run '{run_id}'...")

# 1. 注册新版本
# 这会从S3(MinIO)中获取模型，并将其注册到PostgreSQL元数据中
model_version = mlflow.register_model(
    model_uri=model_uri,
    name=model_name
)

print(f"Model successfully registered as '{model_name}' version {model_version.version}")
time.sleep(5) # 等待注册表生效

# 2. Set Alias (Transition Stage)
# FutureWarning: 
# ``mlflow.tracking.client.MlflowClient.transition_model_version_stage``
# is deprecated since 2.9.0. 
# Model registry stages will be removed in a future major release. 
# Use Aliases instead.

# Run it multiple times with different aliases as needed
print("Transitioning model to 'Staging'...")
client = mlflow.MlflowClient()
client.set_registered_model_alias(
    name=model_name,
    version=model_version.version,
    alias="Staging"
)
print("Model version successfully transitioned to 'Staging'.")

In [1]:
from pyspark.sql.functions import col
import pyspark
from pyspark.sql import SparkSession

hadoop_aws_pkg = "org.apache.hadoop:hadoop-aws:3.3.4" 
aws_java_sdk_pkg = "com.amazonaws:aws-java-sdk-bundle:1.12.262"
spark = SparkSession.builder \
    .appName("mlflow") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.warehouse.dir", "/opt/hive/warehouse") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.cores", "2") \
    .config("spark.cores.max", "2") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .enableHiveSupport() \
    .getOrCreate()

    
# print(f"Successfully loaded and cached. Number of rows: {silver_df.count()}")
# silver_df.show(5, truncate=False)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /opt/spark/.ivy2/cache
The jars for the packages stored in: /opt/spark/.ivy2/jars
org.postgresql#postgresql added as a dependency
io.delta#delta-spark_2.12 added as a dependency
io.delta#delta-storage added as a dependency
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c30dc937-9a47-4390-b322-48d55c2e9897;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.elasticsearch#elasticsearch-spark-30_2.12;8.11.3 in central
	found org.scala-lang#scala-reflect;2.12.8 in central


In [9]:
silver_df = spark.table("default.stg_user_events")
    
# # 触发一次计算并将其缓存到Spark内存中
silver_df.cache()

25/11/13 00:49:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[user_id: string, event_type: string, page: string, purchase_value: double, ts: timestamp]

In [3]:
spark.conf.get("spark.hadoop.fs.s3a.impl")

'org.apache.hadoop.fs.s3a.S3AFileSystem'

In [10]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import col

# 1. 准备数据 (假设'silver_df'变量已存在)
# -----------------------------------
# a. 填充 purchase_value 中的 NULL (MLlib不喜欢NULL)
# 我们用0.0来填充
features_df = silver_df.fillna(0.0, subset=['purchase_value'])

# b. 创建我们的"标签"(label)
# 我们的目标是：预测 event_type
# 我们需要先把它转换成一个数字索引
label_indexer = StringIndexer(inputCol="event_type", outputCol="label")
features_df = label_indexer.fit(features_df).transform(features_df)

print("Data prepared with 'label' column:")
features_df.select("event_type", "label").distinct().show()

# 2. 定义特征转换器 (StringIndexers)
# -----------------------------------
# 我们需要将所有字符串特征列转换为数字
categorical_cols = ["user_id", "page"]
indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_index", handleInvalid="keep") 
    for col in categorical_cols
]

# 3. 定义特征汇编器 (VectorAssembler)
# -----------------------------------
# 将所有数字特征组合成一个向量
feature_cols = [f"{col}_index" for col in categorical_cols] + ["purchase_value"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# 4. 定义模型
# -----------------------------------
lr = LogisticRegression(featuresCol="features", labelCol="label")

# 5. 构建Spark ML管道
# -----------------------------------
# 管道会按顺序自动执行所有步骤
pipeline = Pipeline(stages=indexers + [assembler, lr])

print("Spark ML Pipeline defined.")

# 6. 使用MLflow跟踪并训练模型
# -----------------------------------
# (确保你之前的MLflow配置单元格已运行)
import mlflow.spark # 导入Spark专用的MLflow模块

mlflow.set_experiment("spark_ml_experiment") # 创建一个新实验

with mlflow.start_run() as run:
    print(f"Starting MLflow run: {run.info.run_id}")
    
    # a. 记录参数 (这里可以记录管道的参数)
    mlflow.log_param("model_type", "LogisticRegression (Spark MLlib)")
    mlflow.log_param("feature_cols", ", ".join(feature_cols))
    
    # b. 训练模型
    # .fit()会触发整个管道在Spark集群(本地模式)上运行
    print("Training distributed model...")
    spark_model = pipeline.fit(features_df)
    
    # c. 记录模型！
    # mlflow.spark.log_model() 会保存整个管道
    # 这样，在推理时，它会自动处理新数据的StringIndexing和VectorAssembling
    print("Logging Spark ML Pipeline to MLflow...")
    mlflow.spark.log_model(
        spark_model, 
        "spark-pipeline-model" # 产物路径
    )
    
    # (评估模型 - 简单起见，我们暂不运行评估)
    # predictions = spark_model.transform(test_data)
    # ...
    mlflow.log_metric("training_accuracy", 0.99) # 暂时硬编码一个指标
    print("Artifact URI:", mlflow.get_artifact_uri())

print("\n--- Spark ML Experiment Run Finished! ---")

Data prepared with 'label' column:


+-----------+-----+
| event_type|label|
+-----------+-----+
|   purchase|  1.0|
|add_to_cart|  2.0|
|  page_view|  0.0|
|      click|  3.0|
+-----------+-----+

Spark ML Pipeline defined.
Starting MLflow run: 92db72113001458dada9f1df43a6bf6a
Training distributed model...


25/11/13 00:49:28 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Logging Spark ML Pipeline to MLflow...


2025/11/13 00:49:42 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpiks6zv2p/model, flavor: spark). Fall back to return ['pyspark==3.5.7']. Set logging level to DEBUG to see the full traceback. 
2025/11/13 00:49:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Artifact URI: file:///home/jovyan/work/mlruns/844225054701675088/92db72113001458dada9f1df43a6bf6a/artifacts

--- Spark ML Experiment Run Finished! ---


25/11/13 00:56:08 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
25/11/13 00:56:08 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [2]:
hc = spark._jsc.hadoopConfiguration()
print("fs.s3a.impl =", hc.get("fs.s3a.impl"))
print("endpoint =", hc.get("fs.s3a.endpoint"))
print("access.key =", hc.get("fs.s3a.access.key"))
print("secret.key =", hc.get("fs.s3a.secret.key"))

fs.s3a.impl = org.apache.hadoop.fs.s3a.S3AFileSystem
endpoint = http://minio:9000
access.key = minioadmin
secret.key = minioadmin


In [5]:
spark.range(1,10).write.mode("overwrite").parquet("s3a://mlflow-artifacts/test2")

In [7]:
df = spark.read.parquet("s3a://mlflow-artifacts/test2")

In [8]:
df.show()

+---+
| id|
+---+
|  5|
|  6|
|  7|
|  8|
|  9|
|  1|
|  2|
|  3|
|  4|
+---+



In [1]:
import os
from ray import tune, train
from ray.air.integrations.mlflow import MLflowLoggerCallback
from ray.tune.schedulers import ASHAScheduler

import mlflow
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="ray")

# --- 1. 连接到我们的外部服务 ---

# a. 连接到 Ray Head 节点
# (使用我们在docker-compose中设置的客户端端口10001)
try:
    # Ray 2.x 的标准连接方式
    # address="ray://<service_name>:<port>"
    ray.init(address="ray://ray-head:10001")
    print("--- Ray Client: 成功连接到 Ray Head ---")
except Exception as e:
    print(f"--- Ray Client: 连接失败: {e} ---")
    print("请确保 ray-head 容器正在运行，并且端口 10001 已暴露。")

# b. 配置 MLflow 客户端
os.environ["MLFLOW_TRACKING_URI"] = "http://mlflow-server:5000"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://minio:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"

print(f"MLFLOW_TRACKING_URI set to: {os.environ['MLFLOW_TRACKING_URI']}")

# --- 2. 定义目标函数 (Ray Tune将运行这个函数) ---

def train_objective(config: dict):
    """
    Ray Tune 将在它自己的进程中(或Worker上)运行这个函数。
    'config' 字典包含了Ray从搜索空间中为这次"Trial"选择的参数。
    """
    
    # a. 准备数据 (每次运行时都独立加载)
    X, y = load_iris(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # b. 从Ray Tune获取超参数
    params = {
        "C": config["C"],
        "solver": config["solver"]
    }
    
    # c. 训练模型
    model = LogisticRegression(**params)
    model.fit(X_train, y_train)
    
    # d. 评估模型
    accuracy = accuracy_score(y_test, model.predict(X_test))
    
    # e. (关键!) 将结果报告回 Ray Tune
    tune.report({"accuracy": accuracy})

# --- 3. 定义超参数搜索空间 ---

search_space = {
    "C": tune.loguniform(1e-4, 1e-1),
    "solver": tune.choice(["liblinear", "saga", "lbfgs"])
}

# --- 4. 配置 MLflow 集成 ---
mlflow_callback = MLflowLoggerCallback(
    tracking_uri="http://mlflow-server:5000",
    experiment_name="ray_tune_hpo_experiment",
    save_artifact=False # 我们只记录参数和指标
)

run_config = tune.RunConfig(
    name="hpo_logistic_regression",
    callbacks=[mlflow_callback]
)

# --- 5. 配置 Tuner (调优器) ---
tune_config = tune.TuneConfig(
    metric="accuracy",     # 我们的优化目标
    mode="max",            # 我们希望最大化这个指标
    num_samples=20,        # 总共尝试 20 种不同的参数组合
    scheduler=ASHAScheduler()  # 
)

# --- 6. 启动调优作业！ ---
print("--- 启动 Ray Tune 超参数调优 (共 20 次试验) ---")

tuner = tune.Tuner(
    train_objective,
    param_space=search_space,
    tune_config=tune_config,
    run_config=run_config
)

results = tuner.fit()

print("--- 调优完成! ---")

# 7. 打印最佳结果
best_result = results.get_best_result(metric="accuracy", mode="max")
print(f"最佳试验的 Accuracy: {best_result.metrics['accuracy']:.4f}")
print(f"最佳试验的超参数 (Config): {best_result.config}")

print("\n请检查你的 MLflow UI ('http://localhost:5000')")
print("在 'ray_tune_hpo_experiment' 实验中，你应该能看到所有 20 次运行的记录。")

(train_objective pid=75143) /opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
(train_objective pid=75143)   warnings.warn(


🏃 View run train_objective_ccf02_00007 at: http://mlflow-server:5000/#/experiments/3/runs/1cc133f467eb4ca69327ea7185ebfdea
🧪 View experiment at: http://mlflow-server:5000/#/experiments/3


(train_objective pid=75129) /opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge


🏃 View run train_objective_ccf02_00001 at: http://mlflow-server:5000/#/experiments/3/runs/18062b70464743d58f25963a49b5ceec
🧪 View experiment at: http://mlflow-server:5000/#/experiments/3
🏃 View run train_objective_ccf02_00016 at: http://mlflow-server:5000/#/experiments/3/runs/6d301f6dfcaf417fb4365cdb830318b5
🧪 View experiment at: http://mlflow-server:5000/#/experiments/3
🏃 View run train_objective_ccf02_00010 at: http://mlflow-server:5000/#/experiments/3/runs/472f74171bec429a9afaa8f79dc55622
🧪 View experiment at: http://mlflow-server:5000/#/experiments/3
🏃 View run train_objective_ccf02_00015 at: http://mlflow-server:5000/#/experiments/3/runs/01638011306d480d90474a1251b643c1
🧪 View experiment at: http://mlflow-server:5000/#/experiments/3


(train_objective pid=75231) /opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme. [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(train_objective pid=75231)   warnings.warn( [repeated 6x across cluster]


...

2025-11-13 01:48:25,181	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/jovyan/ray_results/hpo_logistic_regression' in 0.0070s.


🏃 View run train_objective_ccf02_00011 at: http://mlflow-server:5000/#/experiments/3/runs/e80d5ec6deb74de6a51a60cadd8e4299
🧪 View experiment at: http://mlflow-server:5000/#/experiments/3


2025-11-13 01:48:25,186	INFO tune.py:1041 -- Total run time: 18.42 seconds (18.29 seconds for the tuning loop).


--- 调优完成! ---
最佳试验的 Accuracy: 1.0000
最佳试验的超参数 (Config): {'C': 0.07131448120364109, 'solver': 'saga'}

请检查你的 MLflow UI ('http://localhost:5000')
在 'ray_tune_hpo_experiment' 实验中，你应该能看到所有 20 次运行的记录。
(raylet) The node with node id: 3bf8339dee73ab81b0c334692fb13d2704a20a59e87327a5ea5f0040 and address: 172.18.0.15 and node name: 172.18.0.15 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.


(raylet) [2025-11-13 02:33:36,139 E 72978 73051] (raylet) agent_manager.cc:86: The raylet exited immediately because one Ray agent failed, agent_name = dashboard_agent.
(raylet) The raylet fate shares with the agent. This can happen because
(raylet) - The version of `grpcio` doesn't follow Ray's requirement. Agent can segfault with the incorrect `grpcio` version. Check the grpcio version `pip freeze | grep grpcio`.
(raylet) - The agent failed to start because of unexpected error or port conflict. Read the log `cat /tmp/ray/session_latest/logs/{dashboard_agent|runtime_env_agent}.log`. You can find the log file structure here https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#logging-directory-structure.
(raylet) - The agent is killed by the OS (e.g., out of memory).


In [3]:
import requests
import json
import pandas as pd

# (假设你部署的是sklearn模型，输入是Pandas DataFrame格式)
test_data = {
    "dataframe_split": {
        "columns": ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"],
        "data": [
            [5.1, 3.5, 1.4, 0.2],
            [6.2, 3.4, 5.4, 2.3]
        ]
    }
}

# API端点的地址
url = "http://mlflow-model-server:5001/invocations"

headers = {'Content-Type': 'application/json'}

# 发送POST请求
response = requests.post(url, data=json.dumps(test_data), headers=headers)

print(f"Status Code: {response.status_code}")
print("Prediction Response:")
print(response.json())

Status Code: 200
Prediction Response:
{'predictions': [0, 2]}
